In [1]:
import numpy as np
import maxflow

### Graph building


Given: A 3D voxel grid
- For each node:
  - Iterate over 6 neighbouring nodes
    - Get midpoint between two nodes
    - Calculate M closest cameras to midpoint
    - Calculate photoconsistency (roh)with M closest cameras
    - Set node weight to ((h^2)*4*pi/3 ) * roh
    - where h is the length of a voxel
  - connect each node to the source, with a weight of lambda*h^3
  
- For the outer nodes:
  - Connect each to the sink, with an infinite weight

In [5]:

camera_positions = [] # List for each camera position
                    # T (or -T ?) for each transformation mtrix
images = [] # All images

## TODO check if these are the correct coordinates for x/y/z directions
x_structure = np.zeros((3,3,3))
x_structure[1][1][2] = 1

y_structure = np.zeros((3,3,3))
y_structure[1][2][1] = 1

z_structure = np.zeros((3,3,3))
z_structure[2][1][1] = 1

def get_pixel_from_3D(camera, coordinate):
    #TODO millin probably has this
    pass

def get_M_closest_cameras(coordinate, M):
    d = [None for _ in range(len(camera_positions))]
    for i in range(len(camera_positions)):
        d[i] = np.norm(coordinate-camera_positions[i])
    sorted_d = sorted(range(len(d)), key=lambda k: d[k])
    return sorted_d[:M]

def create_edge_weights(visual_hull):
    #NOTE: this function isn't used yet but could be an optimization later
    x_plus = np.zeros(visual_hull.shape)
    y_plus = np.zeros(visual_hull.shape)
    z_plus = np.zeros(visual_hull.shape)
    
    hull_indices = np.where(visual_hull > 0)
    
    x_plus[hull_indices] = list_of_weights
    
    
def get_variance(pixels):
    ave_R = sum(pixels[:,0])/len(pixels[:,0])
    ave_G = sum(pixels[:,1])/len(pixels[:,1])
    ave_B = sum(pixels[:,2])/len(pixels[:,2])
    ave_pixel = np.array([ave_R, ave_G, ave_B])
    variance = 0
    for i in range(len(pixels)):
        variance += np.norm(pixels[i]-ave_pixel)**2
    return variance

def is_edge(visual_hull, index):
    x,y,z = visual_hull.shape
    if index[0] == 0 || index[1] == 0 || index[2] == 0:
        return True
    if index[0]+1 == x || index[1]+1 == y || index[2]+1 == z:
        return True
    neighbours = []
    index = np.array(index)
    neighbours.append(index + np.array([1,0,0]))
    neighbours.append(index + np.array([-1,0,0]))
    neighbours.append(index + np.array([0,1,0]))
    neighbours.append(index + np.array([0,-1,0]))
    neighbours.append(index + np.array([0,0,1]))
    neighbours.append(index + np.array([0,0,-1]))
    
    for n in neighbours:
        if visual_hull[n[0],n[1],n[2]] == 0:
            return True
        
    return False
    

def create_graph(visual_hull, M=3, el=10, h=0.1, origin=[0,0,0]):
    # M: closest cameras
    # el: lambda value
    # h: size of voxel
    # origin: What the index [0,0,0] in the voxel grid corresponds to in 3D coordinate system
    
    
    graph = maxflow.GraphFloat()
    node_ids = graph.add_grid_nodes(visual_hull.shape)
    hull_indices = np.where(visual_hull > 0)
    valid_node_ids = node_ids[hull_indices]

    hull_coords = np.array([hull_indices[0]*h + origin[0], hull_indices[1]*h + origin[1], hull_indices[2]*h + origin[2]])
    
#     edge_coordinates = []
#     edge_coordinates.append(np.array([hull_coords[0]+0.5*h,hull_coords[1],hull_coords[2]]))
#    # edge_coordinates.append(np.array([hull_coords[0]-0.5*h,hull_coords[1],hull_coords[2]]))
#     edge_coordinates.append(np.array([hull_coords[0],hull_coords[1]+0.5*h,hull_coords[2]]))
#    # edge_coordinates.append(np.array([hull_coords[0],hull_coords[1]-0.5*h,hull_coords[2]]))
#     edge_coordinates.append(np.array([hull_coords[0],hull_coords[1],hull_coords[2]+0.5*h]))
   # edge_coordinates.append(np.array([hull_coords[0],hull_coords[1],hull_coords[2]-0.5*h]))
    
    
    # Indices of the 6 neighbouring nodes for each visual hull node
#     edge_indices = []
#     edge_indices.append(np.array([hull_indices[0]+1,hull_indices[1],hull_indices[2]]))
#     edge_indices.append(np.array([hull_indices[0]-1,hull_indices[1],hull_indices[2]]))
#     edge_indices.append(np.array([hull_indices[0],hull_indices[1]+1,hull_indices[2]]))
#     edge_indices.append(np.array([hull_indices[0],hull_indices[1]-1,hull_indices[2]]))
#     edge_indices.append(np.array([hull_indices[0],hull_indices[1],hull_indices[2]+1]))
#     edge_indices.append(np.array([hull_indices[0],hull_indices[1],hull_indices[2]-1]))
    
    for i in range(len(hull_indices[0])):
        #current_index = np.array(zip(hull_indices)[i])
        current_index = np.array([hull_indices[0][i],hull_indices[1][i],hull_indices[2][i]])
        #coordinate = np.array(zip(hull_coords)[i])
        coordinate = np.array([hull_coords[0][i],hull_coords[1][i],hull_coords[2][i]])
        closest_cameras = get_M_closest_cameras(coordinate,M)
        node_id = valid_node_ids[i]
        
        x_coord = coordinate + np.array([0.5*h, 0, 0])
        y_coord = coordinate + np.array([0, 0.5*h, 0])
        z_coord = coordinate + np.array([0, 0, 0.5*h])
        
        pixels_x = []
        pixels_y = []
        pixels_z = []
        for camera in closest_cameras:
            pixels_x.append(get_pixel_from_3D(camera, x_coord))
            pixels_y.append(get_pixel_from_3D(camera, y_coord))
            pixels_z.append(get_pixel_from_3D(camera, z_coord))
        
        x_weight = get_variance(pixels_x)
        y_weight = get_variance(pixels_y)
        z_weight = get_variance(pixels_z)

        # TODO maybe check if the edge is being connected to a node not in the visual hull; i.e. don't add that edge
        
        # N-Links
        graph.add_grid_edges(np.array([node_id]), weights=np.array([x_weight]), structure=x_structure, symmetric=True)
        graph.add_grid_edges(np.array([node_id]), weights=np.array([y_weight]), structure=y_structure, symmetric=True)
        graph.add_grid_edges(np.array([node_id]), weights=np.array([z_weight]), structure=z_structure, symmetric=True)
        
        
        # Source Link
        if is_edge(visual_hull, current_index):
            graph.add_tedge(node_id, el*(h**3), 999999999)
        else:
            graph.add_tedge(node_id, el*(h**3), 0)
        
    return graph, node_ids
    
    
def run_maxflow(graph, node_ids):
    maxflow_value = graph.maxflow()
    segments = graph.get_grid_segments(node_ids)
    return maxflow_value, segments
    
    
    

In [2]:
a = np.array([[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]]])

In [11]:
np.array([5])

array([5])

In [25]:
zip([np.where(a>4)[0],np.where(a>4)[1],np.where(a>4)[2]] )

[(array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]),),
 (array([1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2]),),
 (array([1, 2, 0, 1, 2, 1, 2, 0, 1, 2, 1, 2, 0, 1, 2]),)]

In [19]:
np.zeros(a.shape)

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [9]:
a.shape


(3, 3, 3)

In [12]:
a

array([[[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]],

       [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]],

       [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]])

In [13]:
a[np.array([1,1,1])]

array([[[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]],

       [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]],

       [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]])

In [14]:
np.array([1,1,1])

array([1, 1, 1])

In [20]:
a[[1,1,1]]

SyntaxError: invalid syntax (<ipython-input-20-48aac974df54>, line 1)